In [1]:
import syft as sy # install syft according to https://blog.openmined.org/installing-pysyft-0-5-0rc1-on-a-raspberry-pi-4/https://blog.openmined.org/installing-pysyft-0-5-0rc1-on-a-raspberry-pi-4/
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch as th
import matplotlib.pyplot as plt
import psutil as psu
from time import time, localtime, sleep
import numpy as np
from psutil import cpu_freq
from usage import Usage

In [ ]:
monitor = Usage(interval = 5)

## Sets the name of the saved data, based on device and resource state

In [ ]:
rpi_name = 'RPi4_1000'
test_name = '64bit_WIFI'

## CPU frequency according to experiment (1500, 1000, 600) MHz

In [ ]:
!sudo cpufreq-set --max 1000Mhz
cpu_freq()

Based on Openmined's course: 
https://courses.openmined.org/courses/foundations-of-private-computation

The code for the Worker used in course can be found here: 
https://github.com/OpenMined/courses/blob/foundations-of-private-computation/federated-learning/duet_mnist/MNIST_Syft_Data_Owner.ipynb  

In [ ]:
duet = sy.launch_duet()

## For Training on local data

In [ ]:
mnsit = datasets.load_digits()

In [ ]:
X, y = mnsit.data, mnsit.target

X, _, y, _ = train_test_split(X, y, test_size=0.2, shuffle = False, random_state=42)
#Test data Ignored here

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, mnsit.images, mnsit.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

In [ ]:
X = th.FloatTensor(X)
y = th.FloatTensor(y)

In [ ]:
X = X.tag("digits-data")
y = y.tag("digits-target")

X = X.describe("This is a dataset for handwritten digit classification of " + str(len(X)) + " samples with each sample of size " + str(len(X[0])))
y = y.describe("Labels for digitis (Random State = 42): "+ str(th.unique(y)))

In [ ]:
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)
print('Encryted data sent')

In [ ]:
# See the object stored in the tensor
duet.store.pandas

## When Training on data



In [ ]:
duet.requests.add_handler(action="accept")

In [ ]:
monitor.run_monitor_thread(ipykernel = True, topProc = False)

In [ ]:
usage_data = monitor.stop_monitor_thread()
usage_data.to_csv('data/{}/usage_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))

In [ ]:
!sudo cpufreq-set --max 1500Mhz
cpu_freq()

## Array Access test

In [ ]:
def get_array(store):
    
    indexPtr = -1
    print('Waiting to recieve tensor')
    while len(duet.store.pandas) == 0:
        pass
    print('Tensor recieved')  
    
    for ptr in store:
        indexPtr += 1
        if ptr.tags[0] == 'Send Tensor':
            arrLength = int(ptr.description.split(' ')[-1])
            arr = ptr.get()
            
    return arrLength, arr

In [ ]:
duet.store.pandas

In [ ]:
usage_data = monitor.stop_monitor_thread()
usage_data.to_csv('data/microbench/{}/usage_data_{}_{}.csv'.format(rpi_name,rpi_name,test_name))

In [ ]:
arrLength, arr =  get_array(duet.store)
returnArr = np.empty(shape = (arrLength,3))
index = 0

startTotalAccessTime = time()

print('Starting array access')

array = arr[:,1]

for i in array:
    startAccessTime = time()
    x = arr[int(i),0]
    returnArr[index] = (time(), time() - startAccessTime, float(x))
    index += 1

print('Total array access time:',time()-startTotalAccessTime,'seconds')    
    
returnArr = th.FloatTensor(returnArr)
returnArr = returnArr.tag("Return Tensor")
returnArr = returnArr.describe("This tensor contains the sequenced array with timestamp and access time of shape = ("+ str(arrLength)+",3)")

duet.requests.add_handler(action="accept")

returnArr_ptr = returnArr.send(duet, pointable=True)

duet.requests.add_handler(action="accept")